In [ ]:
import numpy as np
import pandas as pd

# Carga de los datos

In [ ]:
invoice_train = pd.read_csv('Dataset/invoice_train.csv')
client_train = pd.read_csv('Dataset/client_train.csv')
invoice_test = pd.read_csv('Dataset/invoice_test.csv')
client_test = pd.read_csv('Dataset/client_test.csv')

# Datos de entrenamiento

In [ ]:
invoice_train

In [ ]:
client_train

Para reducir el tiempo de procesamiento, se cambiaron todos los "train_client_#" por un simple número #. Con esto se consigue reducir el preprocesamiento de los datos de aproximadamente 70 horas a 5 horas al calcular la media, desviación estándar, min, max y el rango para cada cliente.

Los datos luego de realizar este proceso ya se encuentra guardados en el archivo "invoice_train_2" adjunto a este Notebook, por lo que se puede evitar correr las 4 siguientes celdas de código y directamente ir a cargar el archivo.

In [ ]:
old_client_id = "train_Client_0"
index = 0
for i in range(4476749):
  client_id = invoice_train.loc[i,"client_id"]
  if old_client_id != client_id:
    index += 1
  old_client_id = client_id
  invoice_train.loc[i,"client_id"] = index

In [ ]:
invoice_train = invoice_train.set_axis(invoice_train['client_id'],0)
invoice_train = invoice_train.drop('client_id', axis=1)

Se reemplaza los "ELEC" por 0 y "GAZ" por 1, para trabajar con valores numéricos

In [ ]:
invoice_train["counter_type"] = invoice_train["counter_type"].replace(["ELEC", "GAZ"], [0, 1])

Se elimina la fila de las fechas para reducir el preprocesamiento, ya que se cuenta con otra columna de "months_number".

In [ ]:
invoice_train = invoice_train.drop('invoice_date', axis=1)

Se puede correr la siguiente celda para evitar correr las últimas 4 celdas de código y ahorrar tiempo.

In [ ]:
invoice_train = pd.read_csv('Dataset/invoice_train_2.csv') #se cambio los "train_client_#" por "#" y los Elec por 0 y Gaz por 1

# Datos de testeo 

In [ ]:
invoice_test

In [ ]:
client_test

Se repite este proceso para los datos de testeo. Aquí se cambia los "test_client_#" por un simple número #. Del mismo modo se puede evitar correr las siguientes 4 celdas y directamente cargar el archivo "invoice_test_2" para ahorrar tiempo.

In [ ]:
old_client_id = "test_Client_0"
index = 0
for i in range(1939730):
  client_id = invoice_test.loc[i,"client_id"]
  if old_client_id != client_id:
    index += 1
  old_client_id = client_id
  invoice_test.loc[i,"client_id"] = index

In [ ]:
invoice_test = invoice_test.set_axis(invoice_test['client_id'],0)
invoice_test = invoice_test.drop('client_id', axis=1)

Se reemplaza los "ELEC" por 0 y "GAZ" por 1, para trabajar con valores numéricos

In [ ]:
invoice_test["counter_type"] = invoice_test["counter_type"].replace(["ELEC", "GAZ"], [0, 1])

Se elimina la fila de las fechas para reducir el preprocesamiento, ya que se cuenta con otra columna de "months_number".

In [ ]:
invoice_test = invoice_test.drop('invoice_date', axis=1)

Se puede correr la siguiente celda para evitar correr las últimas 4 celdas de código y ahorrar tiempo.

In [ ]:
invoice_test = pd.read_csv('Dataset/invoice_test_2.csv') #se cambio los "test_client_#" por "#" y los Elec por 0 y Gaz por 1

# Calculo de la media, desviación estándar, mínimo, máximo y rango para los datos de entrenamiento

# Datos de entrenamiento

Se suman las columnas del consumo debido a que presentan una alta correlación y representan el consumo total

In [ ]:
invoice_train["consumo_total"] = invoice_train["consommation_level_1"] + invoice_train["consommation_level_2"] + invoice_train["consommation_level_3"] + invoice_train["consommation_level_4"]
invoice_train = invoice_train.drop(["consommation_level_1","consommation_level_2","consommation_level_3","consommation_level_4"], axis=1)

Se añade una nueva columna para guardar la diferencia entre el `old_index` y el `new_index` que corresponde a la diferencia de consumos entre diferentes facturas

In [ ]:
invoice_train["diff_index"] = invoice_train["new_index"] - invoice_train["old_index"]

Se define un nuevo dataset para guardar las columnas de la media, std, min y max

In [ ]:
col_mean = [col + "_mean" for col in invoice_train.columns]
col_std = [col + "_std" for col in invoice_train.columns]
col_min = [col + "_min" for col in invoice_train.columns]
col_max = [col + "_max" for col in invoice_train.columns]
data_train = pd.DataFrame(columns = col_mean + col_std + col_min + col_max)

Se calcula la media, std, min y max de todas las columnas. Este proceso puede tardar más de 5 horas, por lo que se puede evitar correr las siguientes 3 celdas y cargar directamente el archivo "data_train.csv"

In [ ]:
functions = ['mean','std','min','max']
for index in range(135493):    
    if type(invoice_train.loc[index].count()) != np.int64:
        stats = invoice_train.loc[index].agg(functions,axis=0)
        for function in functions:
            cols = tuple(col + "_" + function for col in invoice_train.columns)
            data_train.loc[index,cols] = stats.loc[function].rename({col: col + "_" + function for col in invoice_train.columns})
    else:
        for function in functions:
            cols = tuple(col + "_" + function for col in invoice_train.columns)
            data_train.loc[index,cols] = dataset.loc[index].rename({col: col + "_" + function for col in invoice_train.columns})
data_train

Teniendo el min y max se calcula la diferencia para determinar el rango de las columnas

In [ ]:
cols_range = ["tarif_type", 
             "counter_number",
             "counter_statue",
             "counter_code",
             "reading_remarque",
             "counter_coefficient",
             "old_index",
             "new_index",
             "months_number",
             "counter_type",
             "consumo_total",
             "diff_index"]

for col in cols_range:
    data_train[col + "_range"] = pd.to_numeric(data_train[col + "_max"], errors='coerce') - pd.to_numeric(data_train[col + "_min"], errors='coerce')

Se unen las columnas de los datos sacados del invoice_train con client_train que contienen datos del cliente como el distrito, la categoría, región y la etiqueta de los datos para entrenamiento

In [ ]:
data_train = pd.concat([data_train, client_train.loc[:,("disrict", "client_catg", "region", "target")]],axis = 1)

Se puede correr la siguiente celda para evitar correr las últimas 3 celdas de código y ahorrar tiempo.

In [ ]:
data_train = pd.read_csv('Dataset/data_train.csv')

# Datos de testeo

Se suman las columnas del consumo debido a que presentan una alta correlación y representan el consumo total

In [ ]:
invoice_test["consumo_total"] = invoice_test["consommation_level_1"] + invoice_test["consommation_level_2"] + invoice_test["consommation_level_3"] + invoice_test["consommation_level_4"]
invoice_test = invoice_test.drop(["consommation_level_1","consommation_level_2","consommation_level_3","consommation_level_4"], axis=1)

Se añade una nueva columna para guardar la diferencia entre el `old_index` y el `new_index` que corresponde a la diferencia de consumos entre diferentes facturas

In [ ]:
invoice_test["diff_index"] = invoice_test["new_index"] - invoice_test["old_index"]

Se define un nuevo dataset para guardar las columnas de la media, std, min y max

In [ ]:
col_mean = [col + "_mean" for col in invoice_test.columns]
col_std = [col + "_std" for col in invoice_test.columns]
col_min = [col + "_min" for col in invoice_test.columns]
col_max = [col + "_max" for col in invoice_test.columns]
data_test = pd.DataFrame(columns = col_mean + col_std + col_min + col_max)

Se calcula la media, std, min y max de todas las columnas. Este proceso puede tardar más de 1 hora, por lo que se puede evitar correr las siguientes 3 celdas y cargar directamente el archivo "data_test.csv"

In [ ]:
functions = ['mean','std','min','max']
for index in range(135493):    
    if type(invoice_test.loc[index].count()) != np.int64:
        stats = invoice_test.loc[index].agg(functions,axis=0)
        for function in functions:
            cols = tuple(col + "_" + function for col in invoice_test.columns)
            data_test.loc[index,cols] = stats.loc[function].rename({col: col + "_" + function for col in invoice_test.columns})
    else:
        for function in functions:
            cols = tuple(col + "_" + function for col in invoice_test.columns)
            data_test.loc[index,cols] = dataset.loc[index].rename({col: col + "_" + function for col in invoice_test.columns})
data_test

Teniendo el min y max se calcula la diferencia para determinar el rango de las columnas

In [ ]:
cols_range = ["tarif_type", 
             "counter_number",
             "counter_statue",
             "counter_code",
             "reading_remarque",
             "counter_coefficient",
             "old_index",
             "new_index",
             "months_number",
             "counter_type",
             "consumo_total",
             "diff_index"]

for col in cols_range:
    data_test[col + "_range"] = pd.to_numeric(data_test[col + "_max"], errors='coerce') - pd.to_numeric(data_test[col + "_min"], errors='coerce')

Se unen las columnas de los datos sacados del invoice_train con client_train que contienen datos del cliente como el distrito, la categoría, región y la etiqueta de los datos para entrenamiento

In [ ]:
data_test = pd.concat([data_test, client_test.loc[:,("disrict", "client_catg", "region")]],axis = 1)

Se puede correr la siguiente celda para evitar correr las últimas 3 celdas de código y ahorrar tiempo.

In [ ]:
data_test = pd.read_csv('Dataset/data_test.csv')